In [1]:
from selenium import webdriver
import time
import pandas as pd
import pymongo as mg

In [2]:
#open chrome browser
options = webdriver.ChromeOptions()
options.add_argument("--window-size=1560,840")
browser = webdriver.Chrome(executable_path='../../../chromedriver.exe', options=options) #webdriver_selenium과 web을 연결해주기위함. 

In [3]:
# url in address window
browser.get('https://www.coupang.com/np/search?component=&q=%EB%88%88+%EC%98%81%EC%96%91%EC%A0%9C&channel=user')
browser.implicitly_wait(10)

In [4]:
client = mg.MongoClient(host='mongodb://localhost:27017')
database = client['project_nutrients']
collection = database['eyes_coupang_review']

In [ ]:
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

column_name = ['product_name', 'review_date', 'review_content', 'review_star', 'review_writer']


i=3  
h=2
review_list = []
review_lists = []
main_window = browser.current_window_handle  # 현재 창의 핸들 저장
from selenium.webdriver.common.keys import Keys
k=2
for go in range(2, 12):
    product_element_path = 'span.btn-page > a:nth-child({})'.format(k)
    product_pagination = browser.find_element_by_css_selector(product_element_path)
    click_product ='.search-product-wrap'
    product_list = browser.find_elements_by_css_selector(click_product)
    for product in product_list:
        product.click()

        browser.implicitly_wait(10)
        print(f"현재 탭 제목: {browser.title}")   
        time.sleep(2)

        # 새로운 탭이 열릴 때까지 대기 (예시: 2개 이상의 창이 열리고, 그 중 하나가 현재 탬과 다른 창일 때까지 대기)
        wait = WebDriverWait(browser, 10)
        new_tab_handle = wait.until(EC.new_window_is_opened)

        # 모든 탬의 핸들 가져오기
        all_tab_handles = browser.window_handles

        # 새로운 탬의 핸들 찾아 선택하기
        for handle in all_tab_handles:
            if handle != main_window:
                new_tab_handle = handle

        # 새로운 탬으로 전환하기
        browser.switch_to.window(new_tab_handle)

        # 필요한 작업 수행 및 정보 가져오기 (예시: 페이지 제목 가져오기)
        print(f"새로운 탭 제목: {browser.title}")   

        try :
            # 상품평 클릭
            browser.find_element_by_css_selector('#prod-review-nav-link').send_keys(Keys.ENTER)
            page = browser.find_element_by_css_selector('.product-tab-review-count').text
            page = page.replace(",", "")  # 쉼표 제거
            page = page.replace("(", "")  # 왼쪽 괄호 제거
            page = page.replace(")", "")  # 오른쪽 괄호 제거

            # 이제 정수로 변환 시도
            try:
                page_int = int(page)
                review_page = page_int / 5
                print(review_page)
            except ValueError as e:
                print("타입 변환 오류:", e)

            if page > 100 :
                for go in range(1, page_int, 1):
                # for go in range(1, 2 ,1):
                    element_path = 'div.sdp-review__article__page.js_reviewArticlePagingContainer > button:nth-child({})'.format(h)
                    pagination = browser.find_element_by_css_selector(element_path)
                    time.sleep(2)
                    try:
                        product = browser.find_element_by_css_selector(' article:nth-child({}) > div.sdp-review__article__list__info > div.sdp-review__article__list__info__product-info__name'.format(i)).text
                    except:
                        product = str()
                    try:
                        star = browser.find_element_by_css_selector('article:nth-child({}) > div.sdp-review__article__list__info > div.sdp-review__article__list__info__product-info > div.sdp-review__article__list__info__product-info__star-gray > div'.format(i)).get_attribute('data-rating')
                    except:
                        star = str() 
                    try:
                        date = browser.find_element_by_css_selector('article:nth-child({}) > div.sdp-review__article__list__info > div.sdp-review__article__list__info__product-info > div.sdp-review__article__list__info__product-info__reg-date'.format(i)).text
                    except:
                        date = str()
                    try:
                        content = browser.find_element_by_css_selector('article:nth-child({}) > div.sdp-review__article__list__review.js_reviewArticleContentContainer > div'.format(i)).text
                    except:
                        content = str()
                    try:
                        writer = browser.find_element_by_css_selector('article:nth-child({}) > div.sdp-review__article__list__info > div.sdp-review__article__list__info__user > span'.format(i)).text
                    except:
                        writer = str()

                    review_list= [product, date, content, star, writer]
                    review_lists.append(review_list)

                    # print(별)
                    i = i + 1        
                    if i == 8 :
                        pagination.click()
                        time.sleep(1)
                        h = h + 1
                        i = 3

                    if h == 12:  # h 값이 12일 때 다시 3으로 초기화
                        element_path = 'div.sdp-review__article__page.js_reviewArticlePagingContainer > button:nth-child({})'.format(h)
                        pagination = browser.find_element_by_css_selector(element_path)
                        h = 2

                        print('실패: {}'.format(element_path))
                        break
                # 저장을 위한 작업 
                df_coupang_liver = pd.DataFrame(data=review_lists, columns=column_name)
                data_dict = df_coupang_liver.to_dict(orient='records')
                collection.insert_many(data_dict)
                review_lists = list()
            else : 
                pass

        except :
            pass

        time.sleep(3)
        # 다시 메인 창으로 전환
        browser.switch_to.window(main_window)
        time.sleep(1)
    product_pagination.click()
    k=k+1
    time.sleep(1)



product_list[0].click()